In [42]:
import importlib
import numpy as np

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier, MyRandomForestClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

import notebookutils
importlib.reload(notebookutils)
import notebookutils as nbutl

# Classification Results

Create the Pytable

In [43]:
league_standing_pytable = MyPyTable()
league_standing_pytable.load_from_file("lib/nhl_leaguestandings.csv")

league_standing_pytable.remove_rows_with_missing_values()

league_standing_pytable.drop_col("ROW")
league_standing_pytable.drop_col("RPt%")
league_standing_pytable.drop_col("OL")
league_standing_pytable.drop_col("T")
league_standing_pytable.drop_col("SRS")

#nbutl.discretize_ptspercent(league_standing_pytable.data, )

key = league_standing_pytable.get_key(league_standing_pytable.column_names, ['PTS%'])
nbutl.discretize_ptspercent(league_standing_pytable.data, key[0])
#nbutl.discretize_srs(league_standing_pytable.data, key[1])

nbutl.cup_level_added(league_standing_pytable)


y_col_name = "FINISH"

y = league_standing_pytable.get_column(y_col_name)
league_standing_pytable.drop_col(y_col_name)
X = [league_standing_pytable.data[i][6:] for i in range(len(league_standing_pytable.data))]


Get the k-stratified folds

In [44]:
train_folds, test_folds = myevaluation.stratified_kfold_cross_validation(X, y, 10)
# convert them all into instances
print("X: ", len(X))
print("y: ", len(y))
print("train folds: ", len(train_folds))
print("test folds: ", len(test_folds))
for i in range(len(train_folds)):
    print(len(train_folds[i]))
    print(len(test_folds[i]))
    # if i == 0: print(test_folds[i])
    # if i == 0: print(train_folds[i])
train_fold_instances = myutils.indices_list_to_instances(train_folds, X)
test_fold_instances = myutils.indices_list_to_instances(test_folds, y)
print("train folds: ", len(train_fold_instances))
print("test folds: ", len(test_fold_instances))
#TODO: discretization needed
print(len(X))


X:  1350
y:  1350
train folds:  10
test folds:  10
1215
135
1215
135
1215
135
1215
135
1215
135
1215
135
1215
135
1215
135
1215
135
1215
135
train folds:  12150
test folds:  1350
1350


In [45]:
# descision tree

descision_tree = MyDecisionTreeClassifier()
descision_tree.fit(X, y)

descision_tree.print_decision_rules(league_standing_pytable.column_names[6:], y_col_name)
descision_tree.visualize_tree(dot_fname="out/tree.dot", pdf_fname="out/tree.pdf", attribute_names=league_standing_pytable.column_names[6:])
print(descision_tree.tree)

IF PLAYOFFS=N THEN FINISH=0.0
IF PLAYOFFS=Y THEN FINISH=1.0


----------------------------------------------


TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [ ]:
random_forest = MyRandomForestClassifier()

# starting Values for forest
N = 10
M = 5
F = 2

train_folds, test_folds = myevaluation.stratified_kfold_cross_validation(X, y, 10)
train_fold_instances = myutils.indices_list_to_instances(train_folds, X)
test_fold_instances = myutils.indices_list_to_instances(test_folds, y)
print("train: ", len(train_fold_instances))
print(type(train_fold_instances))
print("test ", len(test_fold_instances))
print(type(test_fold_instances))

random_forest.fit(train_fold_instances, test_fold_instances, N=10, M=5, F=2)


train:  12150
<class 'list'>
test  1350
<class 'list'>


IndexError: list index out of range